In [5]:
# impoting all the necessary libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import html5lib
import lxml
from datetime import datetime 
from io import StringIO




In [2]:
pip install html5lib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [97]:
# Step 1: Use webdriver to get the URL

url = "https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html"
driver = webdriver.Safari()  
driver.get(url)
# extract text from page
page_content = driver.page_source
# using BeautifulSoup to read/parse the data
soup = BeautifulSoup(page_content, 'html.parser')
# prettify to view the content and find the appropriate table
soup.prettify()
# identifying the appropriate table 
table = soup.find('table')

# using StringIO to convert HTML to file-like object
table_str = str(table)
table_string = StringIO(table_str)

# using Pandas to read the HTML file into a DataFrame
dataframe = pd.read_html(table_string)[0]
# setting names for the columns in the dataframe
dataframe.columns = ['rank', 'x_class', 'date', 'region', 'start_time', 'max_time', 'end_time', 'movie']

# STEP 2

# dropping the last column
dataframe = dataframe.iloc[:, :-1]
# print(dataframe)
# combining date and time into three datetime columns
for index, row in dataframe.iterrows():
    date = row["date"]
    start = row["start_time"]
    end = row["end_time"]
    max = row["max_time"]

    # combining them into objects (new columns)
    start_datetime = datetime.strptime(f"{date} {start}", "%Y/%m/%d %H:%M")
    max_datetime = datetime.strptime(f"{date} {max}", "%Y/%m/%d %H:%M")
    end_datetime = datetime.strptime(f"{date} {end}", "%Y/%m/%d %H:%M")

    # now updating the values in the dataframe
    dataframe.at[index, 'start_datetime'] = start_datetime
    dataframe.at[index, 'end_datetime'] = end_datetime
    dataframe.at[index, 'max_endtime'] = max_datetime

# dropping the original columns from the list 
dataframe = dataframe.drop(['date', 'start_time', 'max_time', 'end_time'], axis=1)

# somehow above process misaligns number of columns so dropping one again
dataframe = dataframe.iloc[:, :-1]
print(dataframe)

# closing the webdriver
driver.quit()

# at this point, the data looks slightly different than in the example but we should be good 


    rank  x_class  region      start_datetime        end_datetime
0      1     X40+     486 2003-11-04 19:29:00 2003-11-04 20:06:00
1      2  X28.57+    9393 2001-04-02 21:32:00 2001-04-02 22:03:00
2      3  X24.57+     486 2003-10-28 09:51:00 2003-10-28 11:24:00
3      4  X24.42+     808 2005-09-07 17:17:00 2005-09-07 18:03:00
4      5  X20.67+    9415 2001-04-15 13:19:00 2001-04-15 13:55:00
5      6   X14.36     486 2003-10-29 20:37:00 2003-10-29 21:01:00
6      7   X13.37    2673 2017-09-06 11:53:00 2017-09-06 12:10:00
7      8   X12.97    8100 1997-11-06 11:49:00 1997-11-06 12:01:00
8      9   X12.95     930 2006-12-05 10:18:00 2006-12-05 10:45:00
9     10   X11.96     486 2003-11-02 17:03:00 2003-11-02 17:39:00
10    11   X11.88    2673 2017-09-10 15:35:00 2017-09-10 16:31:00
11    12   X10.18     720 2005-01-20 06:36:00 2005-01-20 07:26:00
12    13    X9.96    1263 2011-08-09 07:48:00 2011-08-09 08:08:00
13    14     X9.4     930 2006-12-06 18:29:00 2006-12-06 19:00:00
14    15  

In [81]:
# STEP 3 : Scraping the NASA data

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
from io import StringIO
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Set up the WebDriver
url = "http://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html"
driver = webdriver.Safari()
driver.get(url)

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
page_content = driver.page_source

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(page_content, 'html.parser')

# Find the appropriate table within the pre tag
table = soup.find('pre')
# Extract each row of data as a long string
rows = table.text.split('\n')

# Create a list to store dictionaries representing each row
data_list = []

# Process each row and extract data
for row in rows[11:493]:  # Skip the header and footer
    columns = row.split()
    start_date = columns[0]
    year = start_date.split('/')[0]  # Extract the year from the start_date
    end_date = f"{year}/{columns[2]}"  # Add the year to the end_date
    data_dict = {
        'start_date': start_date,
        'start_time': columns[1],
        'end_date': end_date,
        'end_time': columns[3],
        'start_frequency': columns[4],
        'end_frequency': columns[5],
        'flare_location': columns[6],
        'flare_region': columns[7],
        'flare_classification': columns[8],
        'cme_date': columns[9],
        'cme_time': columns[10],
        'cme_angle': columns[11],
        'cme_width': columns[12],
        'cme_speed': columns[13],
        'halo_lower_bound': columns[14]
    }
    data_list.append(data_dict)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Set the index to start from 1
df.index = df.index + 1

# Display the DataFrame
print(df)

    
# Close the webdriver
driver.quit()    


     start_date start_time    end_date end_time start_frequency end_frequency  \
1    1997/01/20      08:56  1997/01/20    09:02           14000         12000   
2    1997/04/01      14:00  1997/04/01    14:15            8000          4000   
3    1997/04/07      14:30  1997/04/07    17:30           11000          1000   
4    1997/05/12      05:15  1997/05/14    16:00           12000            80   
5    1997/05/21      20:20  1997/05/21    22:00            5000           500   
..          ...        ...         ...      ...             ...           ...   
478  2014/09/01      11:12  2014/09/01    20:05           16000           150   
479  2014/09/09      00:05  2014/09/09    13:00           11000           100   
480  2014/09/10      17:45  2014/09/11    12:00           14000           100   
481  2014/09/20      05:10  2014/09/20    05:30           14000          9700   
482  2014/09/22      06:13  2014/09/22    06:50           16000          4900   

    flare_location flare_re

In [82]:
#step 4

# Recode any missing entries as NaN
missing_values = ['----', '-----', '------']
df = df.replace(missing_values, pd.NA)

# Create a new column 'is_halo' that indicates if a row corresponds to a halo flare or not
df.loc[:, 'is_halo'] = df['cme_angle'].eq('Halo')
# Replace Halo entries in the cme_angle column as NA
df['cme_angle'] = df['cme_angle'].replace('Halo', pd.NA)

# Create a new column 'width_lower_bound' that indicates if width is given as a lower bound
df.loc[:, 'width_lower_bound'] = df['cme_width'].apply(lambda x: isinstance(x, str) and x.startswith('>'))
# Remove any non-numeric part of the width column
df.loc[:, 'cme_width'] = pd.to_numeric(df['cme_width'], errors='coerce')

# Combine date and time columns for start, end, and cme
df.loc[:, 'start_datetime'] = pd.to_datetime(df['start_date'] + ' ' + df['start_time'])
df.loc[:, 'end_datetime'] = pd.to_datetime(df['end_date'] + ' ' + df['end_time'], errors='coerce')
df.loc[:, 'cme_datetime'] = pd.to_datetime(df['cme_date'] + ' ' + df['cme_time'], errors='coerce')

# Combine date and time columns
df['start_datetime'] = pd.to_datetime(df['start_date'] + ' ' + df['start_time'])
df['end_datetime'] = pd.to_datetime(df['end_date'] + ' ' + df['end_time'], errors='coerce')
df['cme_datetime'] = pd.to_datetime(df['cme_date'] + ' ' + df['cme_time'], errors='coerce')

# Drop the original date and time columns
df = df.drop(['start_date', 'start_time', 'end_date', 'end_time', 'cme_date', 'cme_time'], axis=1)

# Rename columns
df = df.rename(columns={'flare_classification': 'importance', 'halo_lower_bound': 'plot'})

# Rearrange columns in the DataFrame
df = df[['start_datetime', 'end_datetime', 'start_frequency', 'end_frequency', 'flare_location',
         'flare_region', 'importance', 'cme_datetime', 'cme_angle', 'cme_width', 'cme_speed',
         'plot', 'is_halo', 'width_lower_bound']]

# Convert DataFrame to a formatted string and print
table_str = df.to_string(index=True, col_space=5)
print(table_str)

           start_datetime        end_datetime start_frequency end_frequency flare_location flare_region importance cme_datetime cme_angle cme_width cme_speed  plot  is_halo  width_lower_bound
1     1997-01-20 08:56:00 1997-01-20 09:02:00           14000         12000           <NA>         <NA>       <NA>          NaT       281      72.0       175  PHTX    False              False
2     1997-04-01 14:00:00 1997-04-01 14:15:00            8000          4000         S25E16         8026       M1.3          NaT        74      79.0       312  PHTX    False              False
3     1997-04-07 14:30:00 1997-04-07 17:30:00           11000          1000         S28E19         8027       C6.8          NaT      <NA>     360.0       878  PHTX     True              False
4     1997-05-12 05:15:00 1997-05-14 16:00:00           12000            80         N21W08         8038       C1.3          NaT      <NA>     360.0       464  PHTX     True              False
5     1997-05-21 20:20:00 1997-05-21 22:

/var/folders/2v/c7s82vrd1hs1l3qh9pyjcxy40000gn/T/ipykernel_1217/855049684.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.loc[:, 'cme_datetime'] = pd.to_datetime(df['cme_date'] + ' ' + df['cme_time'], errors='coerce')
/var/folders/2v/c7s82vrd1hs1l3qh9pyjcxy40000gn/T/ipykernel_1217/855049684.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['cme_datetime'] = pd.to_datetime(df['cme_date'] + ' ' + df['cme_time'], errors='coerce')
